# 0. Install

In [1]:
! nvidia-smi

Wed May 27 12:55:48 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
# uncomment if you are running from google colab
! pip install sklearn_crfsuite -q
! pip install https://github.com/PyThaiNLP/pythainlp/archive/dev.zip -q
! pip install fastai2 -q
! pip install emoji -q

     |████████████████████████████████| 747kB 684kB/s 
     \ 22.5MB 124kB/s
     |████████████████████████████████| 194kB 6.9MB/s 
     |████████████████████████████████| 51kB 3.0MB/s 


# 1. Import

In [3]:
#fastai
from fastai2.basics import *
from fastai2.text.all import *

#pythainlp
from pythainlp.ulmfit import *
from pythainlp.tokenize import THAI2FIT_TOKENIZER

Corpus: wiki_lm_lstm
- Downloading: wiki_lm_lstm 0.32


100%|██████████| 1050919089/1050919089 [01:42<00:00, 10281144.47it/s]


Corpus: wiki_itos_lstm
- Downloading: wiki_itos_lstm 0.32


100%|██████████| 1530484/1530484 [00:00<00:00, 5430745.67it/s]


# 2. Dummy Dataset

In [4]:
#get dummy data
imdb = untar_data(URLs.IMDB_SAMPLE)
dummy_df = pd.read_csv(imdb/'texts.csv')

# dummy_df.head()

# 3. Data pipeline

In [0]:
class XThaiTokenizer():
    def __init__(self, lang: str = "th", split_char=' ', **kwargs): 
        self.split_char=split_char
        self.lang = lang

    def __call__(self, items): return (THAI2FIT_TOKENIZER.word_tokenize(t) for t in items)

    def add_special_cases(self, toks):
        pass

In [6]:
#get vocab
thwiki_vocab = pickle.load(open(THWIKI_LSTM['itos_fname'],'rb'))

len(thwiki_vocab), thwiki_vocab[:20]

(60005,
 ['xxunk',
  'xxpad',
  'xxbos',
  'xxfld',
  'xxmaj',
  'xxup',
  'xxrep',
  'xxwrep',
  ' ',
  '\n',
  'ใน',
  'ที่',
  'และ',
  'ของ',
  'เป็น',
  'มี',
  'ได้',
  'การ',
  '"',
  '('])

In [0]:
imdb_lm = DataBlock(blocks=TextBlock.from_df('text', is_lm=True, tok_func=XThaiTokenizer, vocab=thwiki_vocab, sep=''),
                    get_x=ColReader('text'),    
                    splitter=ColSplitter())

In [0]:
# imdb_lm.summary(dummy_df, bs=64, seq_len=72)

In [9]:
dls = imdb_lm.dataloaders(dummy_df, bs=64, seq_len=72)

In [10]:
len(dls.vocab)

60005

In [0]:
# dls.show_batch(max_n=5)

# 4. Model

In [0]:
language_model_learner??
# AWD_LSTM??

In [0]:
config = dict(emb_sz=400, n_hid=1550, n_layers=4, pad_token=1, tie_weights=True, out_bias=True,
             output_p=0.25, hidden_p=0.1, input_p=0.2, embed_p=0.02, weight_p=0.15)

trn_args = dict(drop_mult=0.9, clip=0.12, alpha=2, beta=1)

learn = language_model_learner(dls, AWD_LSTM, config=config, pretrained=False, **trn_args)

# 5. Load Pretrained Model

In [14]:
#load pretrained models
learn.load_pretrained(THWIKI_LSTM['wgts_fname'], THWIKI_LSTM['itos_fname'])

# 6. Generate Text

## ตัวอย่าง 1

In [15]:
print(learn.predict('ตำรวจชุดสืบสวนไปถึงที่เกิดเหตุพบว่า', 200, temperature=0.8, min_p=0.005))

 ตำรวจชุดสืบสวนไปถึงที่เกิดเหตุพบว่า ระเบิดที่ใช้อยู่ในที่เกิดเหตุนั้น อยู่ที่จุดเกิดระเบิด (ระเบิด) ระหว่างการสืบสวน ส่วนตำรวจในกองตำรวจ ที่ถูกจับกุมก็พบว่าระเบิดถูกยิงตกจากหลังรถจนถูกยิง 
 
 ในวันที่ 8 พฤษภาคม พ.ศ. 2548 หน่วยข่าวกรองฝรั่งเศส โดยมี คริส โคลัมบัส หรือ เอลเมอร์ สมิธ เป็นเจ้าหน้าที่ค้นหาข้อมูล และรายงานข่าวว่า มีผู้เสียชีวิตจำนวน 12 คน โดยมีผู้เสียชีวิต 2 คน และบาดเจ็บอีก 1 คน ซึ่งในขณะอยู่ในที่เกิดเหตุ มีทหาร 3 นายที่รอดชีวิตจากเหตุระเบิดในเมือง มีคนหนึ่งเสียชีวิตจากอุบัติเหตุ 
 
 ผลการสอบสวน พบว่า เกิดการยิงกัน ระหว่าง ระเบิด กับ ระเบิด ซึ่งเป็นอาวุธที่ทำให้หน่วยของผู้ก่อการร้าย เสียชีวิต ในข้อหาอาชญากรรมสงคราม โดย เจ้าหน้าที่ CIA และ CIA ได้


## ตัวอย่าง 2

In [16]:
TEXT = "ประชาชนหันมาใช้ถุงมือยางเพื่อ"
N_WORDS = 50
N_SENTENCES = 5

preds = [learn.predict(TEXT, N_WORDS, temperature=0.8, min_p=0.005) 
         for _ in range(N_SENTENCES)]

preds

[' ประชาชนหันมาใช้ถุงมือยางเพื่อทำเป็นเครื่องประดับหรือเครื่องประดับ ซึ่งมีการใช้กันมาอย่างยาวนานตั้งแต่ครั้งสมัยราชวงศ์ฮั่น (ปี ค.ศ. 400 - ค.ศ. 900) โดยมีการแต่งชุดดำเป็นชุด เพื่อแสดงความเคารพและให้ความเคารพต่อ',
 ' ประชาชนหันมาใช้ถุงมือยางเพื่อเป็นอนุสรณ์ และสร้างความสงบสุขให้แก่ชาติ \n \n  \n =ฟุตบอลโลก 2018 รอบคัดเลือก โซนอเมริกาเหนือ – รอบที่ 2= \n ฟุตบอลโลก 2018 รอบคัดเลือก ',
 ' ประชาชนหันมาใช้ถุงมือยางเพื่อดูว่าเป็นใคร แต่เนื่องจากการทำไม้เป็นโลหะและการใช้วัสดุที่มีคุณภาพสูง ทำให้ต้องใช้เงินจำนวนมาก จึงถูกนำไปขายให้กับประชาชน \n \n \n \n \n \n \n ',
 ' ประชาชนหันมาใช้ถุงมือยางเพื่อดูว่าเป็นสีขาวหรือสีขาว มีการตัดสีดำและดำออกจากกัน \n \n การตรวจเยี่ยม. \n วันที่ 18 กันยายน พ.ศ. 25   รัฐบาลรัฐบาลรัฐบาลสหรัฐอเมริกาสหรัฐอเมริกาสหรัฐอเมริกาได้ได้ได้อนุมัติอนุมัติอนุมัติให้ให้ให้มีมีมีการการการตรวจสอบตรวจสอบตรวจสอบ',
 ' ประชาชนหันมาใช้ถุงมือยางเพื่อใช้ในการฝึกและจัดการแสดงในงานวันเสาร์ โดยเริ่มจากการเดิน 1 รอบ ตามถนน โดยผ่านประตูที่ 2 และ 4 จากนั้น เมื่อถึงห้องของผู้ที่มีการถ่ายทอดสด

## ตัวอย่าง 3

In [17]:
TEXT = "เจ้าหน้าที่จึงแจ้งข้อกล่าวหาให้"
N_WORDS = 50
N_SENTENCES = 5

preds = [learn.predict(TEXT, N_WORDS, temperature=0.8, min_p=0.005) 
         for _ in range(N_SENTENCES)]

preds

[' เจ้าหน้าที่จึงแจ้งข้อกล่าวหาให้แก่ บริษัท เอสทีเอ จำกัด (มหาชน) ว่ามีการตรวจสอบหาตัวเท็จ ให้พบว่ามีการใช้ เครื่องตรวจจับ และ รายงานในสภาพที่ผิดพลาด ซึ่งไม่สามารถตรวจสอบได้ โดยการ ตรวจสอบ',
 ' เจ้าหน้าที่จึงแจ้งข้อกล่าวหาให้แก่เจ้าหน้าที่รักษาความปลอดภัยของรัฐบาล ซึ่งเป็นผู้ที่ขอลาออกเพื่อไปเป็นที่ปรึกษาราชการแผ่นดิน ต่อมา ในวันที่ 25 พฤษภาคม พ.ศ. 2547 รัฐบาลได้ออกประกาศ ประกาศ ยกเลิกคำสั่งรัฐบาลที่มีพลเอก ประยุทธ์ จันทร์',
 ' เจ้าหน้าที่จึงแจ้งข้อกล่าวหาให้ลงโทษบุคคลที่เกี่ยวข้องโดยการเผาทรัพย์สินที่เคยใช้ในการคุมขัง โดยการลงโทษ โดยการดำเนินคดี จากคณะกรรมการพิจารณาถึงการทุจริต \n \n ในเดือนพฤศจิกายน ปี พ.ศ. 2550 ศาลพิพากษาให้จำคุก 2',
 ' เจ้าหน้าที่จึงแจ้งข้อกล่าวหาให้ฆ่าตำรวจ ซึ่งอยู่ระหว่างปฏิบัติการ แต่ก็ไม่เป็นผล \n \n เมื่อวันที่ 18 ธันวาคม พ.ศ. 2556 ตำรวจจับกุม นาย วีระยุทธ บุญเกิด สมาชิกสภานิติบัญญัติแห่งชาติ (ประเทศไทย) พ.ศ. ',
 ' เจ้าหน้าที่จึงแจ้งข้อกล่าวหาให้ตำรวจสั่งจับตำรวจที่อำเภอบ้านโป่ง อำเภอปากช่อง จังหวัดประจวบคีรีขันธ์ แต่ก็ไม่เกิดปัญหา ตำรวจจึงได้ให้ตำรวจเข้าจับก

# Credit

* https://www.thainlp.org/pythainlp/tutorials/notebooks/text_generation.html
* http://dev.fast.ai/tutorial.wikitext